In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import ast

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import geopy.distance
from scipy import stats
import category_encoders as ce

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Для распечатки в цвете
from termcolor import colored

import nltk
from nltk.tokenize import word_tokenize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 0. ПОСТАНОВКА ПРОБЛЕМЫ

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# 1. СБОР ДАННЫХ

In [ ]:
# Подгрузим наши данные из соревнования
DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн


In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

# data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Объединяем данные с помощью метода concat
data = pd.concat([df_test, df_train], ignore_index=True, sort=False)


In [ ]:
data.info()

# 2. ОЧИСТКА ДАННЫХ

Поиск пропусков и заполнение данными 


## 2.1. Поиск дубликатов и их удаление 

In [ ]:
dupl_columns = list(data.columns)

mask = data.duplicated(subset=dupl_columns)
data_duplicates = data[mask]
print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')

data_dedupped = data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {data_dedupped.shape[0]}')

## 2.2. Обработка пропущенных значений 



In [ ]:
data.info()

Видно, что пропуски у двух признаков lng и lag - это коорднинаты местоположения гостиницы

In [ ]:
data['hotel_city'] = data['hotel_address'].apply\
(lambda x: x.split()[-5] if x.split()[-1] == 'Kingdom' else x.split()[-2])

In [ ]:
data2 = data.copy()

City_bins = data2.groupby(['hotel_city'])

# найдем медианные координаты с учетом получившихся групп
median_lat = City_bins.lat.median()
median_lng = City_bins.lng.median()

# Применим lambda-функцию к объекту SeriesGroupBy и заменим пропуски соответствующим медианным значением.
data2['lat'] = City_bins.lat.transform(lambda x: x.fillna(x.median()))
data2['lng'] = City_bins.lng.transform(lambda x: x.fillna(x.median()))

data = data2

In [ ]:
def calculate_average_review_score(data):
    # Создаем словарь для хранения средних значений по каждой гостинице
    average_review_scores = {}
    
    # Итерируемся по уникальным названиям гостиниц
    for hotel_name in data['hotel_name'].unique():
        # Выбираем только строки для конкретной гостиницы
        mask = data[data['hotel_name'] == hotel_name]
        # Вычисляем среднее значение между 'average_score' и 'reviewer_score' для гостиницы
        average_score = mask[['average_score', 'reviewer_score']].mean().mean()
        # Сохраняем среднее значение в словаре
        average_review_scores[hotel_name] = average_score
    
    # Добавляем новый столбец 'average_review_score' в основной DataFrame 'data'
    data['average_review_score'] = data['hotel_name'].map(average_review_scores)

# Применяем функцию к основному DataFrame 'data'
calculate_average_review_score(data)


## 2.3. Методы выявления выбросов



#### Пробовал сначала пройтись через метод Тьюки, и с ним получаеться победить выбросы, но когда учу модель измененные признаки становяться не актуальны для модели.

##### Поэтому использую не нормализацию, а стандартизацию признаков и заодно кодирую признаки типа 'object', после изучение модели, данный метод оказался вкуснее всего. 

##### Вначале я конечно же все без исключения признаки закодировал, но когда посмотрел на то, что несет ценность для модели, а что нет, сделал корректировку, поэтому вы можете видеть не похожие по логике признаки. 

### 2.3.1. признак 'additional_number_of_scoring'

некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки

In [ ]:
# Смотрю как распределены значения
data['additional_number_of_scoring'].describe()

In [ ]:
# Знакомлюсь с данными 
data['additional_number_of_scoring'].tail(10)

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='additional_number_of_scoring', ax=axes[0]);
histplot.set_title('Кол-во оценок без отзывов Distribution');
boxplot = sns.boxplot(data=data, x='additional_number_of_scoring', ax=axes[1]);
boxplot.set_title('Кол-во оценок без отзывов Boxplot');

In [ ]:
# Инициализация стандартизатора
scaler = StandardScaler()

# Преобразование признака с помощью стандартизатора
data['standardized_additional_number_of_scoring'] = scaler.fit_transform(data[['additional_number_of_scoring']])


In [ ]:
# Строю график и наблюдаю распределение 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='standardized_additional_number_of_scoring', ax=axes[0]);
histplot.set_title('Стандартизировананное кол-во оценок без отзывов Distribution');
boxplot = sns.boxplot(data=data, x='standardized_additional_number_of_scoring', ax=axes[1]);
boxplot.set_title('Стандартизировананное кол-во оценок без отзывов Boxplot');

In [ ]:
# Наблюдаю стандаритизированные значения 
data['standardized_additional_number_of_scoring'].describe()

### 2.3.2. признак 'total_number_of_reviews'

общее количество действительных отзывов об отеле

In [ ]:
# Смотрю как распределены значения
data['total_number_of_reviews'].describe()

In [ ]:
# Знакомлюсь с данными 
data['total_number_of_reviews'].tail(10)

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='total_number_of_reviews', ax=axes[0]);
histplot.set_title('Количество действительных отзывов Distribution');
boxplot = sns.boxplot(data=data, x='total_number_of_reviews', ax=axes[1]);
boxplot.set_title('Количество действительных отзывов Boxplot');

In [ ]:
# from sklearn.preprocessing import StandardScaler

# Инициализация стандартизатора
scaler = StandardScaler()

# Преобразование признака с помощью стандартизатора
data['standardized_total_number_of_reviews'] = scaler.fit_transform(data[['total_number_of_reviews']].values.reshape(-1, 1))


In [ ]:
# Строю график и наблюдаю распределение 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='standardized_total_number_of_reviews', ax=axes[0]);
histplot.set_title('Стандартизированное количество действительных отзывов Distribution');
boxplot = sns.boxplot(data=data, x='standardized_total_number_of_reviews', ax=axes[1]);
boxplot.set_title('Стандартизированное количество действительных отзывов Boxplot');

In [ ]:
# Наблюдаю стандаритизированные значения 
data['standardized_total_number_of_reviews'].describe()

### 2.3.3. признак 'review_total_negative_word_counts'

общее количество слов в отрицательном отзыве

In [ ]:
# Смотрю как распределены значения
data['review_total_negative_word_counts'].describe()

In [ ]:
# Знакомлюсь с данными 
data['review_total_negative_word_counts'].tail(10)

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='review_total_negative_word_counts', ax=axes[0]);
histplot.set_title('Количество слов в отрицательном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='review_total_negative_word_counts', ax=axes[1]);
boxplot.set_title('Количество слов в отрицательном отзыве Boxplot');

In [ ]:
# Преобразование признака с помощью стандартизатора
data['standardized_review_total_negative_word_counts'] = scaler.fit_transform(data[['review_total_negative_word_counts']])


In [ ]:
# Строю график и наблюдаю распределение 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='standardized_review_total_negative_word_counts', ax=axes[0]);
histplot.set_title('Стандартизированное количество слов в отрицательном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='standardized_review_total_negative_word_counts', ax=axes[1]);
boxplot.set_title('Стандартизированное количество слов в отрицательном отзыве Boxplot');

In [ ]:
# Наблюдаю стандаритизированные значения 
data['standardized_review_total_negative_word_counts'].describe()

### 2.3.4. признак 'review_total_positive_word_counts'

общее количество слов в положительном отзыве

In [ ]:
# Смотрю как распределены значения
data['review_total_positive_word_counts'].describe()

In [ ]:
# Знакомлюсь с данными 
data['review_total_positive_word_counts'].tail(10)

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='review_total_positive_word_counts', ax=axes[0]);
histplot.set_title('Количество слов в положительном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='review_total_positive_word_counts', ax=axes[1]);
boxplot.set_title('Количество слов в положительном отзыве Boxplot');

In [ ]:
data['standardized_review_total_positive_word_counts'] = scaler.fit_transform(data[['review_total_positive_word_counts']])


In [ ]:
# Строю график и наблюдаю распределение 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='standardized_review_total_positive_word_counts', ax=axes[0]);
histplot.set_title('Стандартизированное количество слов в положительном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='standardized_review_total_positive_word_counts', ax=axes[1]);
boxplot.set_title('Стандартизированное количество слов в положительном отзыве Boxplot');

In [ ]:
# Наблюдаю стандаритизированные значения 
data['standardized_review_total_positive_word_counts'].describe()

### 2.3.5. признак 'total_number_of_reviews_reviewer_has_given'

количество отзывов, которые рецензенты дали в прошлом


In [ ]:
# Смотрю как распределены значения
data['total_number_of_reviews_reviewer_has_given'].describe()

In [ ]:
# Знакомлюсь с данными 
data['total_number_of_reviews_reviewer_has_given'].tail(10)

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='total_number_of_reviews_reviewer_has_given', ax=axes[0]);
histplot.set_title('Количество отзывов, которые дали в прошлом Distribution');
boxplot = sns.boxplot(data=data, x='total_number_of_reviews_reviewer_has_given', ax=axes[1]);
boxplot.set_title('Количество отзывов, которые дали в прошлом Boxplot');

In [ ]:
data['standardized_total_number_of_reviews_reviewer_has_given'] = scaler.fit_transform(data[['total_number_of_reviews_reviewer_has_given']])


In [ ]:
# Строю график и наблюдаю распределение 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='standardized_total_number_of_reviews_reviewer_has_given', ax=axes[0]);
histplot.set_title('Стандартизированное количество отзывов, которые дали в прошлом Distribution');
boxplot = sns.boxplot(data=data, x='standardized_total_number_of_reviews_reviewer_has_given', ax=axes[1]);
boxplot.set_title('Стандартизированное количество отзывов, которые дали в прошлом Boxplot');

In [ ]:
# Наблюдаю стандаритизированные значения 
data['standardized_total_number_of_reviews_reviewer_has_given'].describe()

# 3. РАЗВЕДЫВАТЕЛЬНЫЙ АНАЛИЗ ДАННЫХ

## 3.1. Создание новых признаков 


### 3.1.1. Признак 'negative_review'

#### Создание новых признаков, это конечно максимально увлекательное мероприятие, начнем с того, что признак 'negative_review', хранит в себе отзывы и отзывы очень ценны сами по себе, так как каждый человек высказывает свою точку зрения и испытывает при этом определенную эмоцию, такую как положительную, отрицательну и нейтральную. 

##### Построив первый раз признак, я увидел, что признак не несет в себе особоую полезную связь, поэтому я решил категоризировать не только эмоции, но и дополнительно покатели по типу такие как проблемы с едой, гостинецей, окружающей средой, ценой и бред.

In [ ]:
def get_average_score_mean(value):
    if not isinstance(value, str) or not value.strip():
        return 3
    positive_words = ['good', 'excellent', 'wonderful', 'great', 'fantastic', 'awesome', 'superb', 'amazing', 
                      'terrific','no dislikes','nothing to dislike','ok','perfect','faultless','x','clean','cheaper','flawless',
                      'my','yaas']
    negative_words = ['bad', 'poor', 'terrible', 'horrible', 'awful', 'dreadful', 'disappointing', 'unpleasant', 
                      'disappointed','shocking','not much better','decent','did not have','rattled',
                      'not a lot of','could be improved','horrendously smelled','unwilling','unhelpful',
                      'could have been cleaned','chipped','stained','noise','not secure area','easy to be stolen',
                      'not comfortable','disheartened','not advised','everything','leaving','breakfast',
                      'no','na','all','mothing','comfort','old','zero','0','cleaness','above','crowded','dated',
                      'rate','atmosphere','aged','same','scruffy','darky','ele','ba','nothing to dislike']

    tempyra = ['cold','location','parking','view','pool','spa','bar','weather','area']
    room = ['dirty', 'small','hair','carry stuff','staff','pillows','wifi','bathroom','bed','room','gym','elevator',
            'cleaning','reception','restaurant','internet','housekeeping','decor','facilities','shower','service',
            'cleanliness','stuff','furniture','hotel','toilet','lifts','lift','lobby','towels','tv','ac','mattress',
            'carpet','entrance','bathtub','smell','bath']
    food = ['tasted of nothing','stale','dinks','breakfast','food','dinner','coffee','resturant','eggs',
            'cheese','breakfest']
    cost = ['price','expensive','cost','pricing','value']
    
    neutral_words = ['okay', 'average', 'normal', 'standard', 'satisfactory', 'adequate', 'fair','needed', 'found','nothing to complain about',
                     'little problem ','werent sure','apparently','spare bed','nothing to dislike','nothing','no complaints',
                     'nothing really',' ','none','nada','nithing','Nada','na','NA','nil','rien','anything','nulla','null','warmer']

    bred = ['m','a','d','r','s','w','u','adsf','7','2','was','the','h','j','e','z','y','v','c','l','b','mmmm','yu','i',
            'o','nvt','p','t','o','0','g','ras','n','cafe','fees','wc','hmmmmmm']



                      
 # Проверяем тип данных
    if isinstance(value, str):
        # Преобразуем строку в нижний регистр для удобства сравнения
        lower_value = value.strip().lower()

        for word in positive_words:
            if word in lower_value:
                return 1

        for word in negative_words:
            if word in lower_value:
                return 2

        for word in neutral_words:
            if word in lower_value:
                return 3
            
        for word in bred:
            if word in lower_value:
                return 0
            
        for word in tempyra:
            if word in lower_value:
                return 4
            
        for word in room:
            if word in lower_value:
                return 5
            
        for word in cost:
            if word in lower_value:
                return 6
            
    
    # Если ничего не совпало, возвращаем исходное значение
    return value

# Применяем функцию к столбцу 'negative_review' в DataFrame data
data['nega_rew_score'] = data['negative_review'].apply(get_average_score_mean)


In [ ]:
data['nega_rew_score'].describe()

### 3.1.2. Признак 'positive_review'

#### По такой же логике я прохожусь и с признаком 'positive_review'

In [ ]:
def get_average_score_mean(value):
    if not isinstance(value, str) or not value.strip():
        # Если значение не является строкой или является пустой строкой, возвращаем 0 или другое значение,
        # указывающее на отсутствие эмоциональной окраски
        return 3
    # unknown = ['unknown']
    positive_words = ['good', 'excellent', 'wonderful', 'great', 'fantastic', 'awesome', 'superb', 'amazing', 
                      'terrific','no dislikes','nothing to dislike','ok','perfect','faultless','x','clean','cheaper','flawless',
                      'my','yaas']
    negative_words = ['bad', 'poor', 'terrible', 'horrible', 'awful', 'dreadful', 'disappointing', 'unpleasant', 
                      'disappointed','shocking','not much better','decent','did not have','rattled',
                      'not a lot of','could be improved','horrendously smelled','unwilling','unhelpful',
                      'could have been cleaned','chipped','stained','noise','not secure area','easy to be stolen',
                      'not comfortable','disheartened','not advised','everything','leaving','breakfast',
                      'no','na','all','mothing','comfort','old','zero','0','cleaness','above','crowded','dated',
                      'rate','atmosphere','aged','same','scruffy','darky','ele','ba','nothing to dislike']

    tempyra = ['cold','location','parking','view','pool','spa','bar','weather','area']
    room = ['dirty', 'small','hair','carry stuff','staff','pillows','wifi','bathroom','bed','room','gym','elevator',
            'cleaning','reception','restaurant','internet','housekeeping','decor','facilities','shower','service',
            'cleanliness','stuff','furniture','hotel','toilet','lifts','lift','lobby','towels','tv','ac','mattress',
            'carpet','entrance','bathtub','smell','bath']
    food = ['tasted of nothing','stale','dinks','breakfast','food','dinner','coffee','resturant','eggs',
            'cheese','breakfest']
    cost = ['price','expensive','cost','pricing','value']
    
    neutral_words = ['okay', 'average', 'normal', 'standard', 'satisfactory', 'adequate', 'fair','needed', 'found','nothing to complain about',
                     'little problem ','werent sure','apparently','spare bed','nothing to dislike','nothing','no complaints',
                     'nothing really',' ','none','nada','nithing','Nada','na','NA','nil','rien','anything','nulla','null','warmer']

    bred = ['m','a','d','r','s','w','u','adsf','7','2','was','the','h','j','e','z','y','v','c','l','b','mmmm','yu','i',
            'o','nvt','p','t','o','0','g','ras','n','cafe','fees','wc','hmmmmmm','9','k','ffff','3','98','8','5']



                      
 # Проверяем тип данных
    if isinstance(value, str):
        # Преобразуем строку в нижний регистр для удобства сравнения
        lower_value = value.strip().lower()

        for word in positive_words:
            if word in lower_value:
                return 1

        for word in negative_words:
            if word in lower_value:
                return 2

        for word in neutral_words:
            if word in lower_value:
                return 3
            
        for word in bred:
            if word in lower_value:
                return 0
            
        for word in tempyra:
            if word in lower_value:
                return 4
            
        for word in room:
            if word in lower_value:
                return 5
            
        for word in cost:
            if word in lower_value:
                return 6
            
    
    # Если ничего не совпало, возвращаем исходное значение
    return value

# Применяем функцию к столбцу 'negative_review' в DataFrame data
data['posi_rew_score'] = data['positive_review'].apply(get_average_score_mean)


In [ ]:
data['posi_rew_score'].describe()

### 3.1.3. Признак 'mean_posi_neg'

#### Далее я решил объединить два категориальных признака, усредних их между собой. 

##### ДА, я понял, что это была пустая траты времени, так как ничего хорошего из этого не вышло 

In [ ]:
data['mean_posi_neg'] = (data['posi_rew_score'] + data['nega_rew_score']) / 2

## 3.2. Разделение существующих признаков 

### 3.2.1. признак 'hotel_address'



#### Знаю, что в адресе храниться индекс, улица, страна и город. Когда отдельно создавал 4 признака, увидил, что ничего полезного из этого не выходит, поэтому оставляю страна - город, как отдельный самостоятельный признак 



In [ ]:
# Паттерн для извлечения возможных названий стран из строки
country_pattern = r'\b[A-Z][a-z]*(?: [A-Z][a-z]*)*\b'

# Функция для извлечения страны из строки адреса
def extract_country(address):
    countries_found = re.findall(country_pattern, address)
    if countries_found:
        return countries_found[-1]  # Возвращаем последнее найденное название страны
    else:
        return None

# Создание нового признака 'country' в словаре data
data['country'] = [extract_country(address) for address in data['hotel_address']]


#### Смотрю на зависимость признака 

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='country',y='reviewer_score'
            )
axes.set(xlabel='Оценка без отзыва', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости оценки без отзыва')
axes.set_ylim(6, 10)

##### Какой-то явной зависиомости тут нет. 

### 3.2.2. признак 'tags'



#### Один из самых удобных признаков 

##### В этом признаке храниться такая информация как вид поездки, отправка отзыва через устройство, статус гостя, количество прожитых ночей в гостинице, далее количество и вид комнат в номере

 

In [ ]:
# очистим строку oт лишних символов
def tagsclean(tags):
    tags = tags.replace('[', '')
    tags = tags.replace(']', '')
    tags = tags.replace("'", '')
    
    return tags.split(',')
    
data['tags_clean'] = data['tags'].apply(tagsclean)

In [ ]:
dict_tags = {}

for list_tag in data['tags']:
    list_tags = ast.literal_eval(list_tag)
    
    if type(list_tags) is list:
       for tag in list_tags:
           if tag.strip() in dict_tags:
              dict_tags[tag.strip()] += 1
           else:
              dict_tags[tag.strip()] = 1
sorted_tuple = sorted(dict_tags.items(), key=lambda x: x[1],reverse=True)
dict_tags_sort =  dict(sorted_tuple)



#### 3.2.2.1. признак 'trip'

#### Начнем с самого часто встречающегося - вида поездки.

In [ ]:
# Выделим тип поездки из поля тэгов 

def trip(tags):
    
    for curr_tag in tags:
        
        if 'trip' in curr_tag.strip():
            return curr_tag.strip()
        
    return 'Not a trip'

data['trip'] = data['tags_clean'].apply(trip)

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='trip',y='reviewer_score'
            )
axes.set(xlabel='Вид поездки', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от вида поездки')
axes.set_ylim(6, 10)

#### Тип поездки плюс минус у всех одинаковый, ну может чуть больше у тех людей, которые приехали в отпуск 

#### 3.2.2.2. признак 'mobile'

#### Создаем признак по способу отправки отзыва, через мобильное или не мобильное устройство 

In [ ]:
def mobile(tags):
    
    for curr_tag in tags:
        if 'mobile' in curr_tag:
            return 1
    return 0
data['mobile'] = data['tags_clean'].apply(mobile)


In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='mobile',y='reviewer_score'
            )
axes.set(xlabel='Тип устройства', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от типа устройства')
axes.set_ylim(6, 10)

#### Данный признак кажется не интересный для модели, так как значение очень схожи 

#### 3.2.2.3. признак 'guest_stat'

#### Создаем признак который условно называем статусом гостя 

In [ ]:
def guest_status(tags):
    status = ['Couple', 'Solo traveler', 'Group', 'Family with young children',
              'Family with older children', 'Travelers with friends', 'With a pet']
    
    for curr_stat in status:
        if curr_stat in tags:
            return curr_stat
        
    return None

data['guest_stat'] = data['tags'].apply(guest_status)

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='guest_stat',y='reviewer_score'
            )
axes.set(xlabel='Статус гостя', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от статуса гостя')
axes.set_ylim(6, 10)

#### И этот признак, особо не вызывает интереса, хотя можно выделить те группы, где два и более человек ехали одновременно 

#### 3.2.2.4. признак 'night_quantity'

#### Создаем признак количество ночей проживания 

In [ ]:
# import re

def extract_night_quantity(tag):
    # Используем регулярное выражение для поиска числа в строке
    match = re.search(r'\b\d+\b', tag)
    if match:
        return match.group()  # Возвращаем найденное число
    else:
        return '0'  # Если число не найдено, возвращаем 0

def night_quant(tags):
    for tag in tags:
        if 'Stayed' in tag:
            return extract_night_quantity(tag)
    return '0'

data['night_quantity'] = data['tags_clean'].apply(night_quant)


In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='night_quantity',y='reviewer_score'
            )
axes.set(xlabel='Кол-во ночей проживания', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-во ночей проживания')
axes.set_ylim(4, 10)

#### Я думаю, что этот признак с количеством ночей, будет куда более интересен, так как на графике видны различного рода перепады.

#### 3.2.2.5. признак 'rooms'

#### Создаем признак характеристика комнаты 

In [ ]:
def room_def(tags):
    
    for room in tags:
        if 'Room' in room:
            return room
        if 'Suite' in room:
            return room
    return 'Unknown'

data['rooms'] = data['tags_clean'].apply(room_def)

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='rooms',y='reviewer_score'
            )
axes.set(xlabel='Характеристика комнаты ', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от характеристики комнаты ')
axes.set_ylim(6, 10)

#### Что тут можно сказть - какая-то каша, но это и классно. Признак очень интересный :)



### 3.2.3. Разбор признака 'review_date'

#### Перевожу существующий признак времени во все возможные форматы дат для определения зависимости 

In [ ]:
# Преобразуем столбец review_date в datetime и создадаю признаки день, месяц, год, и день недели. 
data['review_date']=pd.to_datetime(data['review_date'])
data['day'] = data['review_date'].dt.day
data['month'] = data['review_date'].dt.month
data['year'] = data['review_date'].dt.year
data['dayofweek'] = data['review_date'].dt.dayofweek

#### 3.2.3.1. признак 'day'

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='day',y='reviewer_score'
            )
axes.set(xlabel='Кол-во дней', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва дней ')
axes.set_ylim(6, 10)

#### Тут все выглядит неинтересно 

#### 3.2.3.2. признак 'month'

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='month',y='reviewer_score'
            )
axes.set(xlabel='Кол-во месяцев', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва месяцев ')
axes.set_ylim(6, 10)

#### Тут хоть какие-то перепады видны 

#### 3.2.3.3. признак 'year'

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='year',y='reviewer_score'
            )
axes.set(xlabel='Кол-во лет', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва лет')
axes.set_ylim(6, 10)

#### Думаю, что не интересный признак 

#### 3.2.3.4. признак 'dayofweek'

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='dayofweek',y='reviewer_score'
            )
axes.set(xlabel='Кол-во недель', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва недель ')
axes.set_ylim(6, 10)

### Даты смотрю,все не интересные 

### 3.2.4. Разбор признаков типа 'object'

#### Пробую создать новые признаки методом пересчета символов и количеством слов. Для анализу беру признаки типа 'object'

In [ ]:
# import nltk
# from nltk.tokenize import word_tokenize

# Инициализация NLTK
nltk.download('punkt')

# Функция для подсчета количества слов в отзыве
def count_words(text):
    tokens = word_tokenize(text.lower())  # Токенизация и приведение к нижнему регистру
    return len(tokens)

data['word_count_negative'] = data['negative_review'].apply(count_words)
data['word_count_positive'] = data['positive_review'].apply(count_words)
data['word_count_hotel_name'] = data['hotel_name'].apply(count_words)
data['word_count_hotel_address'] = data['hotel_address'].apply(count_words)
data['word_count_rooms'] = data['rooms'].apply(count_words)

# Функция для подсчета количества символов в отзыве
def count_characters(text):
    return len(text)

data['char_count_negative'] = data['negative_review'].apply(count_characters)
data['char_count_positive'] = data['positive_review'].apply(count_characters)
data['char_count_hotel_name'] = data['hotel_name'].apply(count_characters)
data['char_count_hotel_address'] = data['hotel_address'].apply(count_characters)
data['char_count_rooms'] = data['rooms'].apply(count_characters)

#### 3.2.4.1. Признак 'word_count_negative'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='word_count_negative',y='reviewer_score'
            )
axes.set(xlabel='Кол-во слов в негативном отзыве', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва слов в негативном отзыве ')
axes.set_ylim(6, 10)

#### Чудесный график, который может быть полезным для модели. Уверен, что тут есть выбросы, поэтому по привичному методу буду их стандартизировать

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='word_count_negative', ax=axes[0]);
histplot.set_title('Кол-во слов в негативном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='word_count_negative', ax=axes[1]);
boxplot.set_title('Кол-во слов в негативном отзыве Boxplot');

In [ ]:
data['standardized_word_count_negative'] = scaler.fit_transform(data[['word_count_negative']])



#### Как и ожидалось выбросов тут достаточно, поэтому и стандартизирую 

#### 3.2.4.2. Признак 'word_count_positive'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='word_count_positive',y='reviewer_score'
            )
axes.set(xlabel='Кол-во слов в позитивном отзыве', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва слов в позитивном отзыве')
axes.set_ylim(6, 10)

#### Чудесный график, уверен, что он тоже будет нести полезную информазицю для модели

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='word_count_positive', ax=axes[0]);
histplot.set_title('Кол-во слов в позитивном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='word_count_positive', ax=axes[1]);
boxplot.set_title('Кол-во слов в позитивном отзыве Boxplot');

In [ ]:
data['standardized_word_count_positive'] = scaler.fit_transform(data[['word_count_positive']])



#### Конечно,я проверил его на выбросы и стандартизировал

#### 3.2.4.3. Признак 'word_count_hotel_name'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='word_count_hotel_name',y='reviewer_score'
            )
axes.set(xlabel='Кол-во слов в наименование гостиницы', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва слов в наименование гостиницы')
axes.set_ylim(6, 10)

#### Странный признак, но он имеет свою определенную ценность, поэтому берем его в оборот 

#### 3.2.4.4. Признак 'word_count_hotel_address'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='word_count_hotel_address',y='reviewer_score'
            )
axes.set(xlabel='Кол-во слов в адресе гостиницы', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва слов в адресе гостиницы')
axes.set_ylim(6, 10)

#### Адрес гостиницы, тоже куда более интересней смотриться, чем те же самые даты 

#### 3.2.4.5. Признак 'word_count_rooms'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='word_count_rooms',y='reviewer_score'
            )
axes.set(xlabel='Кол-во слов в наименование комнат', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва слов в наименование комнат')
axes.set_ylim(6, 10)

#### Ну вот, перепады - здорово. Берем для обчения! 

#### 3.2.4.6. Признак 'char_count_negative'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='char_count_negative',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в негативном отзыве', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в негативном отзыве')
axes.set_ylim(6, 10)

#### Ох уж эта странная красота. 

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='char_count_negative', ax=axes[0]);
histplot.set_title('Кол-во символов в негативном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='char_count_negative', ax=axes[1]);
boxplot.set_title('Кол-во символов в негативном отзыве Boxplot');

In [ ]:
data['standardized_char_count_negative'] = scaler.fit_transform(data[['char_count_negative']])



#### ДА-ДА, все такие страшные вещи я буду стандартизировать!!! 

#### 3.2.4.7. Признак 'char_count_positive'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='char_count_positive',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в позитивном отзыве', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в позитивном отзыве')
axes.set_ylim(6, 10)

#### Чудесненько

In [ ]:
# Строю график, для визуализации информации 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=data, x='char_count_positive', ax=axes[0]);
histplot.set_title('Кол-во символов в позитивном отзыве Distribution');
boxplot = sns.boxplot(data=data, x='char_count_positive', ax=axes[1]);
boxplot.set_title('Кол-во символов в позитивном отзыве Boxplot');

In [ ]:
data['standardized_char_count_positive'] = scaler.fit_transform(data[['char_count_positive']])

#### И это я стандартизирую 

#### 3.2.4.8. Признак 'char_count_hotel_name'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='char_count_hotel_name',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в наименование гостиницы', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в наименование гостиницы')
axes.set_ylim(6, 10)


#### Признак с наименованием гостиниц еще красочней раскрылся 

#### 3.2.4.9. Признак 'char_count_hotel_address'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='char_count_hotel_address',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в адресе гостиницы', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в адресе гостиницы')
axes.set_ylim(6, 10)


#### И Адреса гостиниц, теперь выглядят куда более полезней 

#### 3.2.4.10. Признак 'char_count_rooms'



In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='char_count_rooms',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в наименование комнат', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в наименование комнат')
axes.set_ylim(6, 10)


#### Комнаты в гостинице теперь тоже выглядят прекрасно 

### 3.2.5. Признак 'days_since_review'

#### В данном признаке есть слова и цифры, разделю по слову и вытаскиваю только число

In [ ]:
def extract_day(value):
    # Разделение строки по пробелу и извлечение первого элемента, который должен быть числом
    return int(value.split()[0])

# Применение функции к столбцу 'days_since_review' и создание нового столбца 'day_sin_rev_mean'
data['day_sin_rev_count'] = data['days_since_review'].apply(extract_day)


In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='day_sin_rev_count',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в наименование комнат', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в наименование комнат')
axes.set_ylim(6, 10)


### 3.2.6. Опять признак rooms 

#### Оказывается если признаку присвоить уникальное число, то на обучение столбец смотриться привлекательно 

In [ ]:
# Определение функции get_average_score_mean
def get_average_score_mean(value):
    value = value.strip().lower()
    
    if value not in get_average_score_mean.unique_values:
        get_average_score_mean.unique_values[value] = len(get_average_score_mean.unique_values) + 1
    
    return get_average_score_mean.unique_values[value]

# Инициализация словаря для хранения уникальных значений
get_average_score_mean.unique_values = {}

# Применение функции к столбцу 'rooms' и сохранение результата в новом столбце 'rooms'
data['rooms'] = data['rooms'].apply(get_average_score_mean)

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='rooms',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в наименование комнат', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в наименование комнат')
axes.set_ylim(6, 10)


In [ ]:
data['standardized_rooms'] = scaler.fit_transform(data[['rooms']])

In [ ]:
fig, axes = plt.subplots(1, figsize=(18, 5))
plt.subplots_adjust(wspace=0.6)
sns.barplot(data[data['sample']==1],x='standardized_rooms',y='reviewer_score'
            )
axes.set(xlabel='Кол-во символов в наименование комнат', ylabel='Средняя оценка')
axes.xaxis.set_tick_params(rotation = 30)
axes.set_title('Средняя оценка в зависимости от кол-ва символов в наименование комнат')
axes.set_ylim(6, 10)


### 3.2.7. Признак 'night_quantity'

##### Тут просто меняю тип данных. Странно, что он изночально не поменялся сам 

In [ ]:
data['night_quantity']=data['night_quantity'].astype('int32')

In [ ]:
data.info()

##### Базывые признаки - убираю 

In [ ]:
# Удаляю
data = data.drop(['tags', 'tags_clean', 'review_date','days_since_review'], axis = 1)

# 4. ЭТАП МОДЕЛИРОВАНИЯ И ВЫВОДА МОДЕЛИ В ПРОДАКШЕН

## 4.1. Кодирование признаков

In [ ]:
data.info()

In [ ]:
encoder = ce.OneHotEncoder(cols=['country'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['country'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
encoder = ce.OneHotEncoder(cols=['trip'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['trip'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
encoder = ce.OneHotEncoder(cols=['guest_stat'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['guest_stat'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
data.info()

In [ ]:
data = data.drop(['country', 'trip', 'guest_stat'], axis=1)
 

## 4.2. Данные подводим к продакшену

In [ ]:
data.nunique(dropna=False)

In [ ]:
data.head(1)

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif

# Пример обработки пропущенных значений
# Заполнение пропущенных значений нулями
data.fillna(0, inplace=True)

# Выделение целевой переменной и признаков
y = data.query('sample == 1').drop(['sample'], axis=1).reviewer_score.values
X = data.query('sample == 1').drop(['sample', 'reviewer_score'], axis=1)

# Визуализация результатов анализа значимости
imp_num = pd.Series(f_classif(X[X.columns], y)[0], index=X.columns)
imp_num.sort_values(inplace=True)

fig5, ax5 = plt.subplots(figsize=(15, 20))
imp_num.plot(kind='barh', color='green');

In [ ]:
# import numpy as np

# Задайте порог корреляции
threshold = 0.8

# Вычислите матрицу корреляции
corr_matrix = data.drop(['sample'], axis=1).corr()

# Создайте маску для верхнего треугольника матрицы корреляции (без диагонали)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Выберите пары признаков с корреляцией выше заданного порога
high_correlation_pairs = [(corr_matrix.columns[i], corr_matrix.columns[j]) for i, j in zip(*np.where(np.abs(corr_matrix) > threshold)) if i < j]

# Выведите пары признаков с высокой корреляцией
print("Пары признаков с корреляцией выше", threshold, ":")
for pair in high_correlation_pairs:
    print(pair)


In [ ]:
# Version 27 06.04.24 | MAPE: 0.9090790191440128
data = data.drop(['country_Milan Italy', 'country_Paris France','trip_Not a trip',
                  'guest_stat_Family with older children','year'], axis=1)

In [ ]:
# Version 28 06.04.24 | MAPE: 0.9086750946859526
data = data.drop(['guest_stat_Travelers with friends', 'country_Vienna Austria',
                  'country_Amsterdam Netherlands','country_Barcelona Spain','country_United Kingdom'], axis=1)

In [ ]:
data.info()

## 4.3. Обучаем модель

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
data.info()

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(70).plot(kind='barh')


## 4.4. Отправляем данные 

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# Вывод

* скажу честно, лично разработанные признаки, дали меньше результата, чем тупой про счет слов и символов
* признаки, которые имеют высокую коррелицию их как будто тоже можно оставить, так как модель они не портят 
* я хочу оставить все как есть, так как это мой первый опыт и я считаю, что достаточно много получил для себя инсайтов. Далее я буду пробовать еще лучше разбирать каждый признак 
